In [1]:
#import cac thu vieb can thiet
from tensorflow import keras
from tensorflow.keras import layers
import os
import numpy as np
import cv2
from glob import glob
from sklearn import model_selection
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger, TensorBoard
from tensorflow.keras.metrics import Precision, Recall, MeanIoU
import matplotlib.pyplot as plt
from tensorflow.keras.utils import CustomObjectScope
from tqdm import tqdm
from tensorflow.keras.optimizers import Adam, Nadam, SGD

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
cd /content/drive/'My Drive'/xla

/content/drive/My Drive/xla


In [5]:
#parameter
size_unet=256
#Lua chon cac sieu tham so Hyperparameters
batch=8
lr = 1e-5
epochs = 12

In [6]:
#Ham tra ve tap train, tap valid va tap test
def load_data():
    
    #glob giup hoan thien cac duong dan
    train_x=sorted(glob( "train/images/images/*"))
    train_y=sorted(glob( "train/masks/images/*"))

    valid_x=sorted(glob( "valid/images/images/*"))
    valid_y=sorted(glob( "valid/masks/images/*"))

    test_x=sorted(glob( "test/images/images/*"))
    test_y=sorted(glob( "test/masks/images/*"))
    print("So anh trainning {}".format(len(train_y)))
    print("So anh dung de danh gia {}".format(len(valid_x)))
    print("So anh dung de test {}".format(len(test_x)))
    return (train_x, train_y), (valid_x, valid_y), (test_x, test_y)

In [ ]:
load_data()

In [7]:
#Ap dung voi tap val va tap test
#Doc hinh anh sau do chuan hoa
def read_image(path):
    path=path.decode()
    x = cv2.imread(path,cv2.IMREAD_COLOR)
    x = x/255.0
    return x

def read_mask(path):
    path=path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = x/255.0
    #Them cho du 3 chieu
    x = np.expand_dims ( x, axis = -1)
    return x
#Ham chuyen cac anh ve dang tensorflow
def tf_convert(x, y):
    def _convert(x, y):
      x=read_image(x)
      y=read_mask(y)
      return x,y
    #Chuyen sang dang tensor de dua vao mo hinh
    x, y = tf.numpy_function(_convert, [x, y], [tf.float64, tf.float64])
    #Xac nhan lai hinh dang
    x.set_shape([size_unet, size_unet, 3])
    y.set_shape([size_unet, size_unet, 1])
    return x, y
#Chuan bi de du lieu hoat dong hieu qua voi GPU
#Neu khong co viec chuan bi nay GPU se phai doi CPU load xong du lieu gay ton kem thoi gian
#Cach giai quyet la xu ly song song (pipeline) khi GPU xu ly thi CPU tien hanh chuan bi 
#Dieu nay thuc hien bang cach su dung Tensorflow Dataset API

def tf_dataset(x, y, batch=8):
    #Ham from_tensor_slices() để tạo 1 Dataset objects
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    #Sua lai dung dinh dang ta can thong qua ham tf_convert
    dataset = dataset.map(tf_convert)
    #Chia du lieu theo batch da dinh truoc
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    #Train batch qua nhieu epoch
    dataset = dataset.repeat()
    return dataset


In [8]:
#Ham danh gia dice coef
smooth = 1.
def dice_coef(y_true, y_pred):
    y_true_f = tf.keras.layers.Flatten()(y_true)
    y_pred_f = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + smooth)


def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

In [9]:
(train_x,train_y), (valid_x,valid_y),(test_x,test_y)=load_data()

train_dataset = tf_dataset(train_x, train_y, batch=batch)
valid_dataset = tf_dataset(valid_x, valid_y, batch=batch)
test_dataset = tf_dataset(test_x, test_y, batch=batch)

So anh trainning 11200
So anh dung de danh gia 100
So anh dung de test 100


In [ ]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model

def squeeze_excite_block(inputs, ratio=8):
    init = inputs
    channel_axis = -1
    filters = init.shape[channel_axis]
    se_shape = (1, 1, filters)

    se = GlobalAveragePooling2D()(init)
    se = Reshape(se_shape)(se)
    se = Dense(filters // ratio, activation='relu', kernel_initializer='he_normal', use_bias=False)(se)
    se = Dense(filters, activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(se)

    x = Multiply()([init, se])
    return x

def stem_block(x, n_filter, strides):
    x_init = x

    ## Conv 1
    x = Conv2D(n_filter, (3, 3), padding="same", strides=strides)(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Conv2D(n_filter, (3, 3), padding="same")(x)

    ## Shortcut
    s  = Conv2D(n_filter, (1, 1), padding="same", strides=strides)(x_init)
    s = BatchNormalization()(s)

    ## Add
    x = Add()([x, s])
    x = squeeze_excite_block(x)
    return x


def resnet_block(x, n_filter, strides=1):
    x_init = x

    ## Conv 1
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Conv2D(n_filter, (3, 3), padding="same", strides=strides)(x)
    ## Conv 2
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Conv2D(n_filter, (3, 3), padding="same", strides=1)(x)

    ## Shortcut
    s  = Conv2D(n_filter, (1, 1), padding="same", strides=strides)(x_init)
    s = BatchNormalization()(s)

    ## Add
    x = Add()([x, s])
    x = squeeze_excite_block(x)
    return x

def aspp_block(x, num_filters, rate_scale=1):
    x1 = Conv2D(num_filters, (3, 3), dilation_rate=(6 * rate_scale, 6 * rate_scale), padding="same")(x)
    x1 = BatchNormalization()(x1)

    x2 = Conv2D(num_filters, (3, 3), dilation_rate=(12 * rate_scale, 12 * rate_scale), padding="same")(x)
    x2 = BatchNormalization()(x2)

    x3 = Conv2D(num_filters, (3, 3), dilation_rate=(18 * rate_scale, 18 * rate_scale), padding="same")(x)
    x3 = BatchNormalization()(x3)

    x4 = Conv2D(num_filters, (3, 3), padding="same")(x)
    x4 = BatchNormalization()(x4)

    y = Add()([x1, x2, x3, x4])
    y = Conv2D(num_filters, (1, 1), padding="same")(y)
    return y

def attetion_block(g, x):
    """
        g: Output of Parallel Encoder block
        x: Output of Previous Decoder block
    """

    filters = x.shape[-1]

    g_conv = BatchNormalization()(g)
    g_conv = Activation("relu")(g_conv)
    g_conv = Conv2D(filters, (3, 3), padding="same")(g_conv)

    g_pool = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(g_conv)

    x_conv = BatchNormalization()(x)
    x_conv = Activation("relu")(x_conv)
    x_conv = Conv2D(filters, (3, 3), padding="same")(x_conv)

    gc_sum = Add()([g_pool, x_conv])

    gc_conv = BatchNormalization()(gc_sum)
    gc_conv = Activation("relu")(gc_conv)
    gc_conv = Conv2D(filters, (3, 3), padding="same")(gc_conv)

    gc_mul = Multiply()([gc_conv, x])
    return gc_mul

class ResUnetPlusPlus:
    def __init__(self, input_size=256):
        self.input_size = input_size

    def build_model(self):
        n_filters = [16, 32, 64, 128, 256]
        inputs = Input((self.input_size, self.input_size, 3))

        c0 = inputs
        c1 = stem_block(c0, n_filters[0], strides=1)

        ## Encoder
        c2 = resnet_block(c1, n_filters[1], strides=2)
        c3 = resnet_block(c2, n_filters[2], strides=2)
        c4 = resnet_block(c3, n_filters[3], strides=2)

        ## Bridge
        b1 = aspp_block(c4, n_filters[4])

        ## Decoder
        d1 = attetion_block(c3, b1)
        d1 = UpSampling2D((2, 2))(d1)
        d1 = Concatenate()([d1, c3])
        d1 = resnet_block(d1, n_filters[3])

        d2 = attetion_block(c2, d1)
        d2 = UpSampling2D((2, 2))(d2)
        d2 = Concatenate()([d2, c2])
        d2 = resnet_block(d2, n_filters[2])

        d3 = attetion_block(c1, d2)
        d3 = UpSampling2D((2, 2))(d3)
        d3 = Concatenate()([d3, c1])
        d3 = resnet_block(d3, n_filters[1])

        ## output
        outputs = aspp_block(d3, n_filters[0])
        outputs = Conv2D(1, (1, 1), padding="same")(outputs)
        outputs = Activation("sigmoid")(outputs)

        ## Model
        model = Model(inputs, outputs)
        return model

In [ ]:
arch = ResUnetPlusPlus(input_size=size_unet)
model = arch.build_model()
opt = tf.keras.optimizers.Nadam(lr)
metrics = ["acc",Recall(), Precision(), dice_coef, MeanIoU(num_classes=2)]
model.compile(loss=dice_loss, optimizer=opt, metrics=metrics)

In [ ]:
model.summary()

In [ ]:
#Cac tham so ta se quan sat
callbacks = [
          #Luu bo trong so weight    
          ModelCheckpoint("files/model.h5"),
          #Theo doi loss function va giam LR
          #factor gia tri learning rate se giam 
          #patience: sau so chu ky ma LR se giam
          ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3),
          #Truyen lai ket qua vao CSV
          CSVLogger("files/data.csv"),
          TensorBoard(),
          #Theo doi loss function
          #patience: sau so chu ky neu khong cai thien thi dung
          #restore_best_weights: lua chon khoi phuc tu epoch co ket qua tot nhat
          EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True)
     ]

In [ ]:
train_steps = len(train_x)//batch
valid_steps = len(valid_x)//batch


if len(train_x) % batch != 0:
        train_steps += 1
if len(valid_x) % batch != 0:
        valid_steps += 1

history=model.fit(train_dataset,
validation_data=valid_dataset,
epochs=epochs,
steps_per_epoch=train_steps,
validation_steps=valid_steps,
callbacks=callbacks,
)

In [ ]:
print(history.history.keys())
# summarize history for dice_coef
plt.plot(history.history['dice_coef'])
plt.plot(history.history['val_dice_coef'])
plt.title('Ket qua dice_coef')
plt.ylabel('dice_coef')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for accuracy
plt.plot(history.history['mean_io_u'])
plt.plot(history.history['val_mean_io_u'])
plt.title('Ket qua mean_io_u')
plt.ylabel('mean_io_u')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Ket qua loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [11]:
test_steps = (len(test_x)//batch)
if len(test_x) % batch != 0:
  test_steps += 1
with CustomObjectScope({'dice_loss': dice_loss, 'dice_coef': dice_coef}):
  model = tf.keras.models.load_model("files/model.h5")
model.evaluate(test_dataset, steps=test_steps)

13/13 [==============================] - 7s 381ms/step - loss: 0.2647 - acc: 0.9056 - recall: 0.5723 - precision: 0.9003 - dice_coef: 0.7403 - mean_io_u: 0.7040


[0.2646622359752655,
 0.9055835008621216,
 0.572280764579773,
 0.9003050327301025,
 0.7402838468551636,
 0.703972339630127]

In [12]:
def read_image(path):
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (256, 256))
    x = x/255.0
    return x

def read_mask(path):
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x, (256, 256))
    x = np.expand_dims(x, axis=-1)
    return x

def mask_parse(mask):
    mask = np.squeeze(mask)
    mask = [mask, mask, mask]
    mask = np.transpose(mask, (1, 2, 0))
    return mask

In [13]:
for i, (x, y) in tqdm(enumerate(zip(test_x, test_y)), total=len(test_x)):
        x = read_image(x)
        y = read_mask(y)
        y_pred = model.predict(np.expand_dims(x, axis=0))[0] > 0.5
        h, w, _ = x.shape
        white_line = np.ones((h, 10, 3)) * 255.0

        all_images = [
            x * 255.0, white_line,
            mask_parse(y), white_line,
            mask_parse(y_pred) * 255.0
        ]
        image = np.concatenate(all_images, axis=1)
        cv2.imwrite(f"results/{i}.png", image)

100%|██████████| 100/100 [00:23<00:00,  4.34it/s]
